# Intro

It's time for me to start learning about agents.


## What are agents?

![[Meme/Image source](https://x.com/abhi1thakur/status/1873697074405122144)](imgs/say_agentic_one_more_time.jpeg)

Let's start with some definitions of agents from different sources.


::: {.callout-note}
## Agent Definition from LangChain Blog Post - [source](https://blog.langchain.dev/what-is-an-agent/)
An AI agent is a system that uses an LLM to decide the control flow of an application.
:::

::: {.callout-note}
## Agent Definition from AWS - [source](https://aws.amazon.com/what-is/ai-agents/)
An artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals.
:::

::: {.callout-note}
## Agent Definition from Chip Huyen's Book "AI Engineering" - [source](https://learning.oreilly.com/library/view/ai-engineering/9781098166298/ch06.html#ch06_agents_1730157386572111)

An **agent** is anything that can perceive its environment and act upon that environment. This means that an **agent** is characterized by the environment it operates in and the set of actions it can perform.
:::



::: {.callout-note}
## Agent Definition from Mongo DB Blog Post - [source](https://www.mongodb.com/resources/basics/artificial-intelligence/ai-agents#what-is-an-ai-agent)

An AI **agent** is a computational entity with an awareness of its environment that’s equipped with faculties that enable perception through input, action through tool use, and cognitive abilities through foundation models backed by long-term and short-term memory.
:::

::: {.callout-note}
## Agent Definition from Anthropic - [source](https://www.anthropic.com/research/building-effective-agents)

**"Agent"** can be defined in several ways. Some customers define agents as fully autonomous systems that operate independently over extended periods, using various tools to accomplish complex tasks. Others use the term to describe more prescriptive implementations that follow predefined workflows. At Anthropic, we categorize all these variations as **agentic** systems, but draw an important architectural distinction between **workflows** and **agents**:

**Workflows** are systems where LLMs and tools are orchestrated through predefined code paths.

**Agents**, on the other hand, are systems where LLMs dynamically direct their own processes and tool usage, maintaining control over how they accomplish tasks.
:::


::: {.callout-note}
## Agent Definition from Hugging Face Blog Post on `smolagents` - [source](https://huggingface.co/blog/smolagents)

Any efficient system using AI will need to provide LLMs some kind of access to the real world: for instance the possibility to call a search tool to get external information, or to act on certain programs in order to solve a task. In other words, LLMs should have agency. Agentic programs are the gateway to the outside world for LLMs.


**Agents** are programs where LLM outputs control the workflow. Note that with this definition, **"agent"** is not a discrete, 0 or 1 definition: instead, "agency" evolves on a continuous spectrum, as you give more or less power to the LLM on your workflow
:::


## Is it an Agent? Is it Agentic? It's more like a spectrum with a lot of gray area!

![[tweet from Andrew Ng](https://x.com/AndrewYNg/status/1801295202788983136)](imgs/its_agentic.jpg)

There is a lot of debate and discussion on what exactly is an agent and what is not an agent. I think there is a lot of gray area here and something we 
have to just accept, at least for now. I think Andrew Ng makes some really good points in this [tweet](https://x.com/AndrewYNg/status/1801295202788983136). As Andrew points out, rather than engaging in binary debates about whether something qualifies as a "true agent," we should think about systems as existing on a spectrum of agent-like qualities.  The adjective "agentic" itself becomes particularly useful here, allowing us to describe systems that incorporate agent-like patterns to different degrees without getting caught in restrictive definitions.

This spectrum-based view is reinforced by Anthropic's recent [blog post](https://www.anthropic.com/research/building-effective-agents) on agents. They acknowledge that while they draw an architectural distinction between workflows (systems with predefined code paths) and agents (systems with dynamic control), they categorize both under the broader umbrella of "agentic systems." Similarly, we saw from one of our definitions above that "agent" isn't a discrete, 0 or 1 definition, but rather evolves on a continuous spectrum as you give more or less power to the LLM in your system. This aligns with Andrew Ng's observation that there's a gray zone between what clearly is not an agent (prompting a model once) and what clearly is (an autonomous system that plans, uses tools, and executes multiple steps independently).

![[Nathan Lambert Blog Post on the AI Agent Spectrum](https://www.interconnects.ai/p/the-ai-agent-spectrum)](imgs/llm_loop_is_agent.webp)

Nathan Lambert also writes about the AI agent spectrum in this [blog post](https://www.interconnects.ai/p/the-ai-agent-spectrum). Nathan discusses that the simplest system on this spectrum would be any tool-use language model and that the spectrum of agents increases in complexity from there. I like how Nathan makes the point that the spectrum will continue to evolve and that the definition of an agent will continue to change as the field evolves. Over time, certain technologies will reach milestones where they become definitive examples of AI agents. Therefore at some point, basic tool use with an LLM may not be considered an agent, even though it's the basic starting point on the agentic spectrum.

![](imgs/llm_workflow_vs_loop.png)

Personally, agents and agentic workflows are still so new to me and I have a lot to learn on this topic. I have deployed LLMs in production as well as built some applications where LLMs use function calling (tools) within a conversational chat interface. So I think some of my previous work has fallen somewhere within this AI agentic spectrum, even if it's at one end of the spectrum. I'm going to  keep an open mind and avoid getting caught up in debates about categorical definitions. I'll try to avoid the hype and marketing fluff but be on the lookout for innovation and practical applications. 



# The Tool Calling Loop - The Foundation of Agentic Systems

![[meme source](https://x.com/abhi1thakur/status/1875159964785987904)](imgs/just_loops_meme.jpeg)




In [ ]:
import dotenv

dotenv.load_dotenv()

: 

: 

In [1]:
import json
from concurrent import futures
from typing import Any, Callable, Dict

from litellm import completion
from tools import TOOL_LKP, TOOLS


def call_tool(tool: Callable, tool_args: Dict) -> Any:
    return tool(**tool_args)


def run_step(messages):
    response = completion(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        tools=TOOLS,
    )
    response_message = response.choices[0].message.model_dump()  # TODO: model_dump
    response_message.pop("function_call", None)
    tool_calls = response_message.get("tool_calls", [])
    messages.append(response_message)

    if not tool_calls:
        response_message.pop("tool_calls", None)
        return messages

    tools_args_list = [json.loads(t["function"]["arguments"]) for t in tool_calls]
    tools_callables = [TOOL_LKP[t["function"]["name"]] for t in tool_calls]
    tasks = [(tools_callables[i], tools_args_list[i]) for i in range(len(tool_calls))]

    with futures.ThreadPoolExecutor(max_workers=10) as executor:
        tool_results = list(executor.map(lambda p: call_tool(p[0], p[1]), tasks))

    for tool_call, tool_result in zip(tool_calls, tool_results):
        messages.append(
            {
                "tool_call_id": tool_call["id"],
                "role": "tool",
                "content": str(tool_result),
            }
        )
    return messages


def tool_calling_loop(messages, max_steps=10):
    for counter in range(max_steps):
        print(f"ITERATION {counter=}")
        messages = run_step(messages)
        if messages[-1]["role"] == "assistant" and messages[-1]["content"]:  # TODO
            break
    return messages

/Users/christopher/personal_projects/DrChrisLevy.github.io/posts/agents/env/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:
messages = [
    {"role": "user", "content": "Hello friend!"},
]
tool_calling_loop(messages)

ITERATION counter=0


[{'role': 'user', 'content': 'Hello friend!'},
 {'content': 'Hello! How can I assist you today?', 'role': 'assistant'}]

In [3]:
messages = [
    {"role": "system", "content": "You are a helpful assistant. Use the supplied tools to assist the user. Todays date is 2025-01-03."},
    {
        "role": "user",
        "content": "Recently Steph Curry made 3 pointers in one game without missing. How many did he make? What was the score of the game? Calculate the percentage of the two teams combined points were made by Steph Curry. Also how old is Lebron James and how many points did he score on Jan 2 2025?",
    },
]
tool_calling_loop(messages)

ITERATION counter=0
ITERATION counter=1
ITERATION counter=2


[{'role': 'system',
  'content': 'You are a helpful assistant. Use the supplied tools to assist the user. Todays date is 2025-01-03.'},
 {'role': 'user',
  'content': 'Recently Steph Curry made 3 pointers in one game without missing. How many did he make? What was the score of the game? Calculate the percentage of the two teams combined points were made by Steph Curry. Also how old is Lebron James and how many points did he score on Jan 2 2025?'},
 {'content': None,
  'role': 'assistant',
  'tool_calls': [{'function': {'arguments': '{"query": "Steph Curry 3 pointers made in one game without missing January 2025"}',
     'name': 'web_search'},
    'id': 'call_1ytPzVp1ymTRhr8anFucj64c',
    'type': 'function'},
   {'function': {'arguments': '{"query": "LeBron James age January 2025"}',
     'name': 'web_search'},
    'id': 'call_isgJuv0uCf3AVsoJA10qm1XY',
    'type': 'function'},
   {'function': {'arguments': '{"query": "LeBron James points scored January 2 2025"}',
     'name': 'web_sea

# TODO Thoughts

- the idea of agent writing code instead of just JSON. As discussed in the [Hugging Face Blog Post on `smolagents`](https://huggingface.co/blog/smolagents) as well as the paper [Executable Code Actions Elicit Better LLM Agents](https://huggingface.co/papers/2402.01030)
- HF collection of papers on agents - [source](https://huggingface.co/collections/m-ric/agents-65ba776fbd9e29f771c07d4e)
- [ReAct Paper - main paper on agents?](https://huggingface.co/papers/2210.03629)
- [Secure Code Execution with E2B](https://huggingface.co/docs/smolagents/tutorials/secure_code_execution)
    - But I would like to look into Modal Sand Box too
- [Which AI Agent framework should i use? (CrewAI, Langgraph, Majestic-one and pure code)](https://medium.com/@aydinKerem/which-ai-agent-framework-i-should-use-crewai-langgraph-majestic-one-and-pure-code-e16a6e4d9252)
- [321 real-world gen AI use cases from the world's leading organizations](https://cloud.google.com/transform/101-real-world-generative-ai-use-cases-from-industry-leaders)
    - maybe some interesting ideas here for agents?
- a single LLM call is not agentic!
- an augmented LLM with tools may not be an agent but rather on the agentic spectrum, according to anthropic blog post.
- one cool thing about multi agents is that each agent is modular and can be good at one thing. Also these agents can be different models/prompts/providers etc.
- from CrewAI - role playing, focus, tools, cooperation, guardrails, memory
    - role playing - give the agent a role/persona with system prompt, careful consideration of role and backstory for the agent
    - focus - Too much tools, too much context, etc. Make agent focus on one thing. Don't let agent do too much. 
    - tools - too many tools or overload of tools can lead to issues. Choose the tools carefully for each agent and not too many.
    - cooperation - multi agent cooperation. Just like human can intteract with ChatGPT in iterative manner to get better results/outputs, the LLM agents can interact with each other anc accept feedback, delegate tasks.
    - guardrails - agents getting stuck doing loops, derailing, etc. Many frameworks have guardrails built in.
    - memory - ability for the agent to remember what it has done, what it has learned, etc. Short term memory is mmemory during execution of a task. share knowledge, activities. and learnings with agents. sharing intermedia information even before task completion. Long term memory - after the crew is finished. Stored in DB. LOng term memory can be used in future tasks. Leads to self improving agents.
    - process i.e. process.hierarchical
- CrewAI advice:
    - think like a manager 
    - Cache tool responses across all agents 
    - Tools should be versatile and fault tolerant 
    - I feel like this is simply good SWE practice.


# Frameworks and Platforms

- crewAI
- Langgraph
- Smolagents
- autogen
- [Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/)

# Resources

[OpenAI Note Book - Prerequisite to Swarm - Orchestrating Agents: Routines and Handoffs](https://github.com/openai/openai-cookbook/blob/main/examples/Orchestrating_agents.ipynb)

[Anthropic Blog - Building effective agents](https://docs.anthropic.com/en/docs/build/agents/building-effective-agentshttps://www.anthropic.com/research/building-effective-agents)

[AI Engineering Book - By Chip Huyen](https://learning.oreilly.com/library/view/ai-engineering/9781098166298/)

[Hugging Face Blog Post - Introducing smolagents, a simple library to build agents](https://huggingface.co/blog/smolagents)

[litelllm - GIthub](https://github.com/BerriAI/litellm)

[Executable Code Actions Elicit Better LLM Agents](https://huggingface.co/papers/2402.01030)

[Hugging Face Collection of Papers on Agents](https://huggingface.co/collections/m-ric/agents-65ba776fbd9e29f771c07d4e)

[Deep Learning AI Course -  Multi AI Agent Systems with crewAI](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai/)


[Raising the bar on SWE-bench Verified with Claude 3.5 Sonnet](https://www.anthropic.com/research/swe-bench-sonnet)

[Building Effective Agents Cookbook](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents)

[deeplearning.ai - The Batch - Issue 281 - Recap of 2024 - with some agentic thoughts](https://www.deeplearning.ai/the-batch/issue-281/)

[LangChain Blog Post - What is an Agent?](https://blog.langchain.dev/what-is-an-agent/)


[Lindy](https://www.lindy.ai/)


["Say Agentic One More Time" Meme](https://x.com/abhi1thakur/status/1873697074405122144)


[AWS: What is an Agent?](https://aws.amazon.com/what-is/ai-agents/)

[Chapter 6 Agents from AI Engineering Book by Chip Huyen](https://learning.oreilly.com/library/view/ai-engineering/9781098166298/ch06.html#ch06_agents_1730157386572111)

[tweet from Andrew Ng on AI Agent Spectrum](https://x.com/AndrewYNg/status/1801295202788983136)

[Nathan Lambert Blog Post on the AI Agent Spectrum](https://www.interconnects.ai/p/the-ai-agent-spectrum)

[Lang Chain Academy - AI Agents with LangGraph](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

[Deep Learning AI Course - LangGraph AI Agents](https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/)


[A simple Python implementation of the ReAct pattern for LLMs](https://til.simonwillison.net/llms/python-react-pattern)

[AI Agents That Matter](https://agents.cs.princeton.edu/)

[Demystifying AI Agents: A Guide for Beginners](https://www.mongodb.com/resources/basics/artificial-intelligence/ai-agents)

[Meme From Hamel Husain about agents vs LLM with Function Calling](https://x.com/HamelHusain/status/1867289397898752222)

